# Reading RTDOSE files and resampling to the Referenced Image Grid
---

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from rosamllib.readers import DICOMImageReader, RTDoseReader
from rosamllib.viewers import interactive_image_viewer

In [3]:
# get the paths to the rtdose file and the referenced image
rtdose_path = r"path/to/rtdose/dicom"
ct_path = r"path/to/reference/dicom/image"

In [4]:
# read the dose file and the referenced image
rtdose = RTDoseReader(rtdose_path).read()
ct_image = DICOMImageReader(ct_path).read()

In [ ]:
# the size of the dose array stored in dicom
rtdose.GetSize()

In [ ]:
# get the image array
img = ct_image.get_image_array()
img.shape

In [ ]:
# resample the dose to the referenced image grid
resampled_dose = rtdose.resample_dose_to_image_grid(ct_image)
resampled_dose_array = resampled_dose.get_dose_array()
resampled_dose_array.shape

In [8]:
# apply windowing and leveling for the image
window_level = 50
window_width = 400
min_val = window_level - (window_width / 2)
max_val = window_level + (window_width / 2)

windowed_img = np.clip(img, min_val, max_val)

In [9]:
# display the axial, sagittal and coronal planes for selected slices
axial_indx = 150
coronal_indx = img.shape[1] // 2
sagittal_indx = img.shape[2] // 2

dose_threshold = np.max(resampled_dose_array) / 2

In [ ]:
fig = plt.figure(figsize=(10, 8))

# axial
axial_ax = fig.add_subplot(1, 2, 1)
axial_ax.imshow(windowed_img[axial_indx], cmap='gray')
axial_overlay = np.ma.masked_less(resampled_dose_array[axial_indx], dose_threshold)
axial_ax.imshow(axial_overlay, cmap='jet', alpha=0.5)
axial_ax.set_title('Axial')
axial_ax.axis('off')

# sagittal
sagittal_ax = fig.add_subplot(2, 2, 2)
sagittal_ax.imshow(windowed_img[:,:,sagittal_indx], cmap='gray', origin='lower')
sagittal_overlay = np.ma.masked_less(resampled_dose_array[:,:,sagittal_indx], dose_threshold)
sagittal_ax.imshow(sagittal_overlay, cmap='jet', alpha=0.5, origin='lower')
sagittal_ax.set_title('Sagittal')
sagittal_ax.axis('off')

# coronal
coronal_ax = fig.add_subplot(2, 2, 4)
coronal_ax.imshow(windowed_img[:,coronal_indx,:], cmap='gray', origin='lower')
coronal_overlay = np.ma.masked_less(resampled_dose_array[:,coronal_indx,:], dose_threshold)
coronal_ax.imshow(coronal_overlay, cmap='jet', alpha=0.5, origin='lower')
coronal_ax.set_title('Coronal')
coronal_ax.axis('off')

plt.tight_layout()
plt.show()

In [11]:
# generate DRR in the coronal and sagittal planes and overlay with dose
coronal_img = np.sum(windowed_img, axis=1)
sagittal_img = np.sum(windowed_img, axis=2)
coronal_dose = np.sum(np.ma.masked_less(resampled_dose_array, dose_threshold), axis=1)
sagittal_dose = np.sum(np.ma.masked_less(resampled_dose_array, dose_threshold), axis=2)

In [ ]:
plt.imshow(coronal_img, cmap='gray', origin='lower')
plt.imshow(coronal_dose, cmap='jet', alpha=0.5, origin='lower')
plt.axis('off')
plt.title('DRR coronal')
plt.show()

In [ ]:
plt.imshow(sagittal_img, cmap='gray', origin='lower')
plt.imshow(sagittal_dose, cmap='jet', alpha=0.5, origin='lower')
plt.axis('off')
plt.title('DRR sagittal')
plt.show()

In [ ]:
# use the interactive display
interactive_image_viewer(ct_image, dose_array=resampled_dose_array, dose_units=resampled_dose.DoseUnits, cmap='gray')